In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dat = pd.read_csv('../input/ecommerce-data/data.csv',encoding="ISO-8859-1")
# ,encoding='cp874'

In [ ]:
dat.head(5)
dff = dat.copy()
dat.head(10)


In [ ]:
dat.head(5)
dff = dat.copy()
dat.head(10)


In [ ]:
duplicated = dff.duplicated().sum()
print(duplicated)
dff.drop_duplicates(inplace= True)

In [ ]:
dff.shape

In [ ]:
#transfore the data type
dff['InvoiceDate']= pd.to_datetime(dff['InvoiceDate'])

In [ ]:
dff.describe()

In [ ]:
#deleting negative values from unit price and quantity
#we have negtive value for quantity and price
df = dff[(dff['Quantity']>0) & (dff['UnitPrice']>0)]

In [ ]:
df.shape
df.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.scatter(df['Quantity'], df['UnitPrice'])
ax.set_xlabel('Quantity')
ax.set_ylabel('UnitPrice')
plt.show()

### Relation of unit price with quantity

In [ ]:
#Taking out outliers to get better plot
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(df[['Quantity','UnitPrice']]))
df = df[(z < 3).all(axis=1)]

In [ ]:
df = df[(df['Quantity']>=0) | (df['UnitPrice']>=0)]

In [ ]:
sns.boxplot(df['Quantity'])

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.scatter(df['Quantity'], df['UnitPrice'])
ax.set_xlabel('Quantity')
ax.set_ylabel('UnitPrice')
plt.show()

In [ ]:
#Checking the missings
df.isna().sum()

In [ ]:
#Creating total amount column
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']


In [ ]:
df['Hour']=df['InvoiceDate'].dt.hour
df['Month']=df['InvoiceDate'].dt.month
df['Weekdays']= df['InvoiceDate'].dt.weekday

### RFM SEGMENTATION

In [ ]:
rfm = df.copy()

In [ ]:
#because we are looking at the customer and not the product we drop InvoiceNo and Stcokcode
ab = df.groupby('CustomerID').agg({'InvoiceDate': 'min','TotalAmount': 'sum'})
ab.rename(columns={'InvoiceDate': "Recency",('InvoiceDate', 'nunique'): "Frequency","TotalAmount": 'Monetization'},  inplace = True)
frequency = df.groupby('CustomerID').agg({'InvoiceDate':'nunique'})
frequency.rename(columns={'InvoiceDate': "Frequency"}, inplace = True)

rfm = pd.merge(ab,frequency, on='CustomerID')

#rfm.rename(columns={('InvoiceDate',     'min'): "Recency",('InvoiceDate', 'nunique'): "Frequency","TotalAmount": 'Monetization'})

In [ ]:
a = rfm.reset_index()

In [ ]:
rfm.rename(columns={'InvoiceDate': "Recency",'InvoiceDate': "Frequency","TotalAmount": 'Monetization'})

In [ ]:
rfm.describe()

In [ ]:
#We save the most recent date to, then , calulate the recency
e = df['InvoiceDate'].min()
print('minimun :'+ str(e))

In [ ]:
rfm.info()

In [ ]:
###create receny, cad calculate the last time he bought something compare to e
rfm['Recency'] = rfm['Recency'].apply(lambda x : (x - e).days)

rfm['BasketSize'] = (rfm['Monetization']/rfm['Frequency']).round(2)

In [ ]:
rfm.describe()

In [ ]:
rfm1 = rfm.copy()
# df.groupby('CustomerID')['InvoiceDate'].max() - df.groupby('CustomerID')['InvoiceDate'].min()
df

In [ ]:
#why do we rank it and how?
rfm['Rank_Recency'] = pd.qcut( rfm['Recency'],q=5, labels = range(6, 1, -1))
rfm['Rank_Recency'] = pd.to_numeric(rfm['Rank_Recency'])

In [ ]:
def freq(x):
    if x ==1:
        return 1
    elif x == 2:
        return 2
    elif x == 3:
        return 3
    elif x == 4:
        return 4
    else: 
        return 5

rfm['Rank_Frequency'] =rfm['Frequency'].apply(freq)

#rfm['Rank_Frequency'] = pd.qcut( rfm['Frequency'],q=5, labels = range(1, 6, 1))
#rfm['Rank_Frequency'] = pd.to_numeric(rfm['Rank_Frequency'])


In [ ]:
rfm['Rank_Monetization'] = pd.qcut( rfm['Monetization'],q=5, labels = range(1, 6, 1))
rfm['Rank_Monetization'] = pd.to_numeric(rfm['Rank_Monetization'])

rfm['RFM_Score'] = rfm['Rank_Recency'].astype(str)+ rfm['Rank_Frequency'].astype(str) + rfm['Rank_Monetization'].astype(str)
rfm['Score'] = rfm['Rank_Recency']+ rfm['Rank_Frequency']+ rfm['Rank_Monetization']

rfm['BasketSize'] = (rfm['Monetization']/rfm['Frequency']).round(2)

In [ ]:
rfm.describe()

In [ ]:
def client_segment(x):
    if x == 15:
        return 'Champions'
    elif  x >= 14:
        return 'Loyal Customers'
    elif  x >= 11:
        return 'Can’t Lose Them'
    elif  x >= 9:
        return 'Potential Loyalist'
    elif  x >= 7:
        return 'Promising'
    elif  x >= 6:
        return 'Needs Attention'
    elif  x >= 5:
        return 'At Risk'
    else:
        return 'Lost'

In [ ]:
rfm['Clients'] = rfm['Score'].apply(client_segment)

In [ ]:
clients = rfm[['Clients', 'Frequency', 'Monetization', 'Recency']].groupby('Clients').median()
clients.reset_index(inplace = True)

In [ ]:
rfm.head(5)

In [ ]:
ax = sns.barplot(x="Frequency", y="Clients", data=clients)
ax.set_ylabel('Clients')
ax.set_title('Median Visits')

In [ ]:
ax = sns.barplot(x="Monetization", y="Clients", data=clients)
ax.set_ylabel('Clients')
ax.set_title('Median Expenditure')

In [ ]:
ax = sns.barplot(x="Recency", y="Clients", data=clients)
ax.set_ylabel('Clients')
ax.set_title('Median time from last shop ')


In [ ]:
import squarify

squarity =rfm['Clients'] .value_counts()
color=['grey','orange','pink','purple', 'brown', 'blue', 'green', 'red']


fig = plt.gcf()
ax = fig.add_subplot()
fig.set_size_inches(12, 8)
squarify.plot(sizes= squarity , 
              label=['Promising',
                     'Can’t Lose Them',
                     'Potential Loyalist',
                     'Loyal Customer',
                     'Promising', 
                     'Needs Attention',
                     'At Risk',
                     'Champions',
                     'Lost',] ,color = color, alpha=0.5,)
plt.title("RFM Segments",fontsize=18,fontweight="bold")
plt.axis('off')
plt.show()

In [ ]:
squarity

Interpretation

We have a deeper understanding of our segmentations from the graphs above, from the median expenditure to each group's number. Among the visualization, we observe three horizontal histograms (median frequency, median monetization and median recency for every group of clients) and one squarity.

As we can see in the Median Expenditure graph, our two best categories of clients are champions (in green) and loyal customer(in purple). As we can see, both clients provide almost 80% of our revenue and only represent 22% of our total clients! This follows the 80/20 rule, also known as the Pareto Principle, attributed to the Italian economist Vilfredo Pareto. In one of his papers, Pareto noted that about 80% of Italy's land belonged to approximately 20% of its total population.

This is a presentation of the three different users:

Champions gather 300 customer who has a median expenditure of $2446 and comes almost every month. For marketers, they should be the most critical clients because they make your market sustainable. In this case, you must regularly reward with the promotion or provide to them privilege access.

Potential Loyalist represents 15% of our customers. They play an essential role in the growth of the market. They already show interest in the brand. The next step is to acquire them by building a relationship.

Customers that need attention constitutes 10% of our market. They spend and come below the average. We translated this with a potential loss, increased churn. It is vital to reactive interest with discounts and recommendations. Moreover, you may also receive feedback to improve your process.

In [ ]:
sns.heatmap(clients.corr(),annot=True)

In [ ]:
import pandas as pd
import numpy as np

rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = clients.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)
# 'RdBu_r' & 'BrBG' are other good diverging colormaps

In [ ]:
# Compute the correlation matrix
corr = clients.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(240, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# sns.heatmap(clients.corr(), annot = True, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')
sns.heatmap(clients.corr(), annot = True, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', linewidths=3, linecolor='black')


In [ ]:
clients

In [ ]:
rfm['Clients'].value_counts()

CLUSTER

Unsupervised is a type of machine learning that concludes from a data set without a target variable. We usually implement this method to find a hidden pattern or understand the structure of the data. It discovers information that may not be visible to the human eye. The most popular method is clustering.

Clustering split your data set into several groups. The group are formed of similar data point compared to the other group. We can use this approach for marketing segmentation, medical imaging, and anomaly detection. They are different types of clustering methods, including:
Partitioning methods
Hierarchical clustering
Fuzzy clustering
Density-based clustering
Model-based clustering
Analogy:

You had a box of apples of different colors; red, green, and mixed colors. Individually picking each color would not only consume time but is likely to entail manifestations of human error. However, with cluster analysis, we can segment these different apples according to color. With such information, one can harvest information such as how many apples belong to which color, which color is of the best quality, or which color has the least quality.

In [ ]:
cluster = rfm.drop(['Rank_Recency','Rank_Frequency','Rank_Monetization','RFM_Score','Score','Clients'], axis = 1)

K-means clustering algorithm
K-means is one of the most straightforward clustering analysis to implement. It attempts to divide the dataset into a fixed number k of a group. The k refers to the number of centroids you want in your model. A centroid is a point at the center of each cluster. The algorithm then allocates every data point to the nearest cluster while keeping it as small as possible.

The appropriate number of clusters is essential for an optimal score. In marketing, a wrong evaluation of it can result in a poor marketing campaign. For example, If you have five kinds of customers, but you only allow them to be segmented into two groups. Your campaign will not target the right audience, which is translated into a low return on investment (ROI)

### Finding Optimal number of clusters

In [ ]:
from sklearn.preprocessing import StandardScaler,  MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering

In [ ]:
X= MinMaxScaler().fit_transform(cluster)
#x = StandardScaler().fit_transform(X)

In [ ]:
sum_of_squared_distances = []
K = range(1,15)
for k in K:
    k_means = KMeans(n_clusters=k)
    model = k_means.fit(X)
    sum_of_squared_distances.append(k_means.inertia_)

In [ ]:
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('sum_of_squared_distances')
plt.title('Elbow Method')
plt.show()

In [ ]:
# Elbow Method for K means
# Import ElbowVisualizer
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(2,30), timings= True)
visualizer.fit(X)        # Fit data to visualizer
visualizer.show()        # Finalize and render figure

In [ ]:
# Silhouette Score for K means
# Import ElbowVisualizer
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(2,30),metric='silhouette', timings= True)
visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
# Calinski Harabasz Score for K means
# Import ElbowVisualizer
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(2,30),metric='calinski_harabasz', timings= True)
visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

from sklearn.metrics import silhouette_score

silhouette_scores = [] 

for n_cluster in range(2, 8):
    silhouette_scores.append( 
        silhouette_score(X, KMeans(n_clusters = n_cluster).fit_predict(X))) 
    
# Plotting a bar graph to compare the results 
k = [2, 3, 4, 5, 6,7] 
plt.bar(k, silhouette_scores) 
plt.xlabel('Number of clusters', fontsize = 10) 
plt.ylabel('Silhouette Score', fontsize = 10) 
plt.show() 

In [ ]:
k_means_5 = KMeans(n_clusters=7)
model = k_means_5.fit(X)
y_hat_5 = k_means_5.predict(X)
labels_5 = k_means_5.labels_
print(metrics.silhouette_score(X, labels_5, metric = 'euclidean'))
metrics.calinski_harabasz_score(X, labels_5)

In [ ]:
cluster['Cluster'] = labels_5

In [ ]:
temp = cluster.reset_index()
print(temp.shape)
# temp.join(dff,on='CustomerID',how='right',lsuffix='left',rsuffix='right')
# pd.concat([dff,temp],join='inner',axis=1)
dff.columns
dff1 = dff.drop(columns=['InvoiceNo', 'StockCode','InvoiceDate'])
print(dff1.shape)
# dff1.join(temp,on='CustomerID',how='left',lsuffix='left',rsuffix='right')

In [ ]:
dff1['CustomerID'] = dff1['CustomerID'].astype(str)
temp['CustomerID'] = temp['CustomerID'].astype(str)
print(dff1.dtypes)
print(temp.dtypes)

### Building classification model to predict customer group

In [ ]:
# temp.join(dff1,on='CustomerID',how='right',lsuffix='left',rsuffix='right')
temp1 = temp.merge(dff1,how='inner',left_on='CustomerID',right_on='CustomerID')
print(temp1.columns)
temp2 = temp1.drop(columns=['Recency', 'Monetization', 'Frequency'])

In [ ]:
temp2
train = temp2.loc[:,temp2.columns != 'Cluster']
target = temp2.loc[:,temp2.columns == 'Cluster']
print(train.shape)
print(target.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['DescriptionCode'] = le.fit_transform(train['Description'])
train['CountryCode'] = le.fit_transform(train['Country'])
train
# train1 = train[['Description','Country']].apply(le.fit_transform)

In [ ]:
temptrain = train.drop(columns=['Description','CustomerID','Country'])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# train[['Quantity1','UnitPrice1']] = scaler.fit_transform(train[['Quantity','UnitPrice']])
scaler.fit(train[['UnitPrice','Quantity']])
scaled_data = pd.DataFrame(scaler.transform(train[['UnitPrice','Quantity']]),columns=['QuantScaled','UnitPriceScaled'])
train1 = pd.concat([train,scaled_data],axis=1)
train2 = train1.drop(columns=['Description','Quantity','UnitPrice','Country'])
train3 = train2.drop(columns=['CustomerID'])
train3

In [ ]:
train3
# train4.drop( )

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temptrain,target,test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train,y_train)

In [ ]:
prediction = clf.predict(X_train)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_train,y_pred=prediction))

In [ ]:
import nltk, warnings
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
from sklearn import metrics
import plotly.graph_objects as go
import plotly.express as px
pd.set_option('display.max_rows', 100)
%config InlineBackend.figure_format = 'svg'
warnings.filterwarnings("ignore")

In [ ]:
df_cleaned = dff.copy(deep = True)
df_cleaned['QuantityCanceled'] = 0

In [ ]:
entry_to_remove = [] ; doubtfull_entry = []

In [ ]:
remaining_entries = df_cleaned[(df_cleaned['Quantity'] < 0) & (df_cleaned['StockCode'] != 'D')]
print("nb of entries to delete: {}".format(remaining_entries.shape[0]))
remaining_entries[:5]

In [ ]:
df_cleaned.drop(remaining_entries.index, axis = 0, inplace = True)

In [ ]:
list_special_codes = df_cleaned[df_cleaned['StockCode'].str.contains('^[a-zA-Z]+', regex=True)]['StockCode'].unique()
list_special_codes

In [ ]:
list_special_codes = df_cleaned[df_cleaned['StockCode'].str.contains('^[a-zA-Z]+', regex=True)]['StockCode'].unique()
list_special_codes

In [ ]:
df_cleaned[(df_cleaned['UnitPrice'] == 0)].head(5)
df_cleaned['TotalPrice'] = df_cleaned['UnitPrice'] * (df_cleaned['Quantity'] - df_cleaned['QuantityCanceled'])

In [ ]:
uk_ecom = df_cleaned[df_cleaned['Country']=='United Kingdom']
uk_ecom['Description'] = uk_ecom['Description'].astype(str)
freq = pd.Series(' '.join(uk_ecom['Description']).split()).value_counts()[:20]
freq

In [ ]:
#Uncommon items
freq1 =  pd.Series(' '.join(uk_ecom['Description']).split()).value_counts()[-20:]
freq1

In [ ]:
stop_words = set(stopwords.words("english"))
new_words = ['RED','PINK', 'BLUE', 'OF', 'BROWN',"BLACK"]
stop_words = stop_words.union(new_words)

In [ ]:
for i in new_words:
  if i in stop_words:
    print(i)

In [ ]:
import re
import nltk.corpus
from nltk import corpus

In [ ]:
corpus = []

In [ ]:
for i in range(0, 8789):
    text = re.sub('[^a-zA-Z]', ' ', uk_ecom['Description'].iloc[i])
    text = text.lower()
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    text = text.split()
    ps=PorterStemmer()
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if word not in stop_words]
    text = " ".join(text)
    i
    corpus.append(text)

In [ ]:
corpus

In [ ]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
wordcloud = WordCloud(    background_color='white',
                          stopwords=stop_words,
                          max_words=200,
                          max_font_size=50, 
                          random_state=42
                         ).generate(str(corpus))

In [ ]:
plt.figure(figsize=(10,5))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Word Cloud for Customer\'s Products')

### Cohort Analysis

In [ ]:
import datetime as dt

In [ ]:
def extract_days(x):
	return dt.datetime(x.year, x.month, x.day)

In [ ]:
uk_ecom['date'] = pd.DatetimeIndex(uk_ecom['InvoiceDate']).date
uk_ecom['InvoiceDay'] = uk_ecom['date'].apply(extract_days)
grouping = uk_ecom.groupby('CustomerID')['InvoiceDay']
uk_ecom['CohortDay'] = grouping.transform('min')
print(uk_ecom.head())

In [ ]:
def extract_month_int(x):
    return dt.datetime(x.year, x.month, 1)


In [ ]:
#create invoice month column
uk_ecom['InvoiceMonth'] = uk_ecom['date'].apply(extract_month_int)
grouping = uk_ecom.groupby('CustomerID')['InvoiceMonth']
uk_ecom['CohortMonth'] = grouping.transform('min')
uk_ecom.head()

In [ ]:
def extract_dates_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

In [ ]:
invoice_year, invoice_month, _ = extract_dates_int(uk_ecom, 'InvoiceMonth')
cohort_year, cohort_month, _ = extract_dates_int(uk_ecom, 'CohortMonth')
years_difference = invoice_year - cohort_year
months_difference = invoice_month - cohort_month


In [ ]:
uk_ecom['CohortIndex'] = years_difference * 12 + months_difference + 1
grouping = uk_ecom.groupby(['CohortMonth', 'CohortIndex'])
cohort_data = grouping['CustomerID'].apply(pd.Series.nunique).reset_index()

In [ ]:
cohort_counts = cohort_data.pivot(index = 'CohortMonth', columns = 'CohortIndex', values = 'CustomerID')
cohort_sizes = cohort_counts.iloc[:, 0]
retention = cohort_counts.divide(cohort_sizes, axis = 0)

In [ ]:
#review the retention table
retention.round(3) * 100
grouping_avg_quantity = uk_ecom.groupby(['CohortMonth', 'CohortIndex'])
cohort_data_avg_quantity = grouping_avg_quantity['Quantity'].mean().reset_index()
average_quantity = cohort_data_avg_quantity.pivot(index = 'CohortMonth', columns = 'CohortIndex', values = 'Quantity')
average_quantity.round(1).fillna('')

In [ ]:
plt.figure(figsize = (10, 5))
plt.title('Retention Rate for Customers in United Kingdom')
sns.heatmap(data = retention, annot = True, fmt = '.0%', vmin = 0.01, vmax = 0.5, cmap = 'BuGn')
plt.show()

## Assosiation Rules(Market Basket Analysis)

In [ ]:
data = dat.copy()

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
data_apriori = data[data['Country']=='United Kingdom']
data_apriori.head()

In [ ]:
# Which Product and Their Count 
data_apr = data_apriori.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
data_apr.head()

In [ ]:
def num(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_new = data_apr.applymap(num)
basket_new.head()

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
rule_fp = fpgrowth(basket_new, min_support=0.02, use_colnames=True)
rule_fp

In [ ]:
items = apriori(basket_new, min_support=0.02, use_colnames=True)
items

In [ ]:
# pd.set_option('display.height', 100)
# pd.set_option('display.max_rows', 100)
pd.options.display.max_rows

In [ ]:
rule = association_rules(items, metric="lift", min_threshold=1)
rule

In [ ]:
association_rules(items, metric="lift", min_threshold=1).head(10)